In [16]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
import re


from google.colab import drive
drive.mount('/content/drive')
gpt2.mount_gdrive()

from nltk.util import ngrams 

from textblob import TextBlob

NGRAM = 2

re_stripper_alpha = re.compile('[^a-zA-Z]+')

sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
from itertools import chain
from collections import Counter
import re
import math

!pip install pipreqs
!pipreqs ./

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ERROR: Failed on file: ./drive/MyDrive/TestFastAPI_v2.py
Traceback (most recent call last):
  File "/usr/local/bin/pipreqs", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/pipreqs/pipreqs.py", line 470, in main
    init(args)
  File "/usr/local/lib/python3.7/dist-packages/pipreqs/pipreqs.py", line 409, in init
    follow_links=follow_links)
  File "/usr/local/lib/python3.7/dist-packages/pipreqs/pipreqs.py", line 138, in get_all_imports
    raise exc
  File "/usr/loca

In [17]:
! cp drive/My\ Drive/JobDescriptionPrediction/Resources/Model/* .
!tar -xvf  'checkpoint_run1.tar' -C .
!tar -xvf  'checkpoint_run2.tar' -C .
!tar -xvf  'checkpoint_run3.tar' -C .
# gpt2.copy_checkpoint_from_gdrive(run_name="/JobDescriptionPrediction/Resources/Model/run1")
# gpt2.copy_checkpoint_from_gdrive(run_name="/JobDescriptionPrediction/Resources/Model/run2")
# gpt2.copy_checkpoint_from_gdrive(run_name="/JobDescriptionPrediction/Resources/Model/run3")

cp: -r not specified; omitting directory 'drive/My Drive/JobDescriptionPrediction/Resources/Model/T5-Aboutcompany'
cp: -r not specified; omitting directory 'drive/My Drive/JobDescriptionPrediction/Resources/Model/T5-Role'
cp: -r not specified; omitting directory 'drive/My Drive/JobDescriptionPrediction/Resources/Model/T5-Skill'
checkpoint/run1/
checkpoint/run1/checkpoint
checkpoint/run1/counter
checkpoint/run1/encoder.json
checkpoint/run1/events.out.tfevents.1617255936.c51996cb59be
checkpoint/run1/hparams.json
checkpoint/run1/model-3337.data-00000-of-00001
checkpoint/run1/model-3337.index
checkpoint/run1/model-3337.meta
checkpoint/run1/vocab.bpe
checkpoint/run2/
checkpoint/run2/checkpoint
checkpoint/run2/counter
checkpoint/run2/encoder.json
checkpoint/run2/events.out.tfevents.1617285150.0c4597a381ce
checkpoint/run2/hparams.json
checkpoint/run2/model-1000.data-00000-of-00001
checkpoint/run2/model-1000.index
checkpoint/run2/model-1000.meta
checkpoint/run2/vocab.bpe
checkpoint/run3/
check

In [18]:
def getModelOutput(Input_Job_title,Input_Skills,Input_Company_name,Input_Location):
  text_skills = run_Model('run1',Input_Job_title+ ' '+Input_Skills)
  
  text_role = run_Model('run2',Input_Job_title+ ' '+Input_Skills)
  
  text_aboutCompany = run_Model('run3',Input_Company_name +' '+Input_Location)
  
  output = 'SKILLS'+'\n'+text_skills + '\n\n'+ 'ROLES'+'\n'+text_role+ '\n\n'+ 'ABOUT COMPANY'+'\n'+text_aboutCompany
  return output

In [19]:
def run_Model(run_name, input):
  tf.reset_default_graph()

  sess = gpt2.start_tf_sess()
  gpt2.load_gpt2(sess, run_name=run_name)
  
  output = gpt2.generate(sess,
                         run_name=run_name,
                  prefix=input,
                  length=100,
                  temperature=0.7,
                  nsamples=1,
                  return_as_list=True
                  )
  output = output[0].replace(input,'')
  output = output.replace('\', \'','')
  output = output.replace('[\'','')
  output = output.replace('[','')
  output = output.replace(']','')
  output = output.replace('.,','.')
  output = output.replace('"','.')
  output = output.replace('"','.')
  output = re.sub('\s+',' ',output)
  output = re.sub('^,','',output)
  print( run_name, ' :: Output generated ')
  return output

In [20]:
cleaned_indeed_data = pd.read_excel('/content/drive/My Drive/JobDescriptionPrediction/Resources/Data/interim/cleaned_indeed_job_dataset.xlsx')

In [21]:
cleaned_indeed_data.columns

Index(['JobID', 'Job_Title', 'Link', 'Queried_Salary', 'Job_Type', 'Skill',
       'No_of_Skills', 'Company', 'No_of_Reviews', 'No_of_Stars',
       'Date_Since_Posted', 'Description', 'Location', 'Company_Revenue',
       'Company_Employees', 'Company_Industry', 'python', 'sql',
       'machine learning', 'r', 'hadoop', 'tableau', 'sas', 'spark', 'java',
       'Others', 'CA', 'NY', 'VA', 'TX', 'MA', 'IL', 'WA', 'MD', 'DC', 'NC',
       'Other_states', 'Consulting and Business Services',
       'Internet and Software', 'Banks and Financial Services', 'Health Care',
       'Insurance', 'Other_industries'],
      dtype='object')

In [22]:
# INPUTS ::
def getInputs(index):
  inputs = []
  inputs.append(cleaned_indeed_data['Job_Title'][index])
  inputs.append(cleaned_indeed_data['Skill'][index])
  inputs.append(cleaned_indeed_data['Company'][index])
  inputs.append(cleaned_indeed_data['Location'][index])
  inputs.append(cleaned_indeed_data['Description'][index])
  return inputs

In [23]:
def tokenizestring(input):
  output = word_tokenize(input)
  output = [word for word in output if not word in stopwords.words()]
  return output

In [24]:
def generateResults(index):  
  inputs = getInputs(index)
  expectedoutput= inputs[4]
  modeloutput = getModelOutput(str(inputs[0]),str(inputs[1]),str(inputs[2]),str(inputs[3]))
  calculateCosineSimilarity(expectedoutput,modeloutput)


def calculateCosineSimilarity(expectedoutput,modeloutput): 

  
  expectedoutput_tokens = tokenizestring(expectedoutput)
  modeloutput_tokens = tokenizestring(modeloutput)


  expectedoutput_keywords = ' '.join(expectedoutput_tokens)
  modeloutput_keywords = ' '.join(modeloutput_tokens)

  
  cos_sim = generateCosineSimilary(modeloutput_keywords,expectedoutput_keywords)

  # print('---------------------------------------------------------------')
  # print('expectedOutput ::' , expectedoutput )
  # print('modeloutput ::' , modeloutput )
  # print('\n\n\n Cosine similarity :: ',cos_sim)
  return cos_sim
def generateCosineSimilary(input1,input2):

  input = [input1,input2]

  input1_vectorizer = CountVectorizer(input)
  input1_vectorizer.fit(input)
  vectors = input1_vectorizer.transform(input).toarray()
  
  cos_lib = cosine_similarity(vectors)
  print('Word similarity :: ',cos_sim)
  return cos_lib[0][1]


In [25]:
def get_tuples_nosentences(txt, NGRAM ):
    """Get tuples that ignores all punctuation (including sentences)."""
    if not txt: return None
    ng = ngrams(re_stripper_alpha.sub(' ', txt).split(), NGRAM)
    return list(ng)
def cosine_similarity_ngrams(a, b):
    vec1 = Counter(a)
    vec2 = Counter(b)
    
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    return float(numerator) / denominator

In [26]:
def getSimilarityMatrix(index):
  res_df = pd.DataFrame()
  df = []
  # res_df = pd.DataFrame(columns = ['Job_Title', 'Skills','Company','Location','Expected Output', 'Model Output','Word_similarity', '2Gram Similarity','4Gram Similarity','6Gram Similarity','8Gram Similarity','10Gram Similarity'])
  inputs = getInputs(index)
  expectedoutput= inputs[4]
  modeloutput = getModelOutput(str(inputs[0]),str(inputs[1]),str(inputs[2]),str(inputs[3]))
  word_similarity  = calculateCosineSimilarity(expectedoutput,modeloutput)
  
  res_df['Job_Title'] = inputs[0]
  res_df['Skills'] = inputs[1]
  res_df['Company']= inputs[2]
  res_df['Location'] = inputs[3]
  res_df['Expected Output'] = inputs[4]
  res_df['Model Output'] = modeloutput
  res_df['Word_similarity']= word_similarity
  df.append(inputs[0])
  df.append(inputs[1])
  df.append(inputs[2])
  df.append(inputs[3])
  df.append(inputs[4])
  df.append(modeloutput)
  df.append(word_similarity)
  print('Word simi completed')
  print(df)
  ngram_sim = []
  loop = [1,2,4,6,8,10]
  for  i in loop:
    expOutputNGram  = get_tuples_nosentences(expectedoutput,2)
    modelOutputNGram  = get_tuples_nosentences(modeloutput,2)
    sim = cosine_similarity_ngrams(expOutputNGram,modelOutputNGram)
    ngram_sim.append(sim)
    df.append(sim)
  res_df['2Gram Similarity'] = ngram_sim[0]
  res_df['4Gram Similarity'] = ngram_sim[1]
  res_df['6Gram Similarity'] = ngram_sim[2]
  res_df['8Gram Similarity'] = ngram_sim[3]
  res_df['10Gram Similarity'] = ngram_sim[4]
  print('n gram completed')


  return df
  


In [27]:
df = getSimilarityMatrix(0)
print(df)

Loading checkpoint checkpoint/run1/model-3337
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-3337
run1  :: Output generated 
Loading checkpoint checkpoint/run2/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run2/model-1000
run2  :: Output generated 
Loading checkpoint checkpoint/run3/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run3/model-1000
run3  :: Output generated 


NameError: ignored

In [ ]:
generateResults(0)
generateResults(10)
generateResults(20)
generateResults(30)
generateResults(40)

In [ ]:
def jaccard_distance(a, b):
    """Calculate the jaccard distance between sets A and B"""
    a = set(a)
    b = set(b)
    return 1.0 * len(a&b)/len(a|b)

In [ ]:
a = get_tuples_nosentences("Above is a bad example of four-gram similarity.")
b = get_tuples_nosentences("This is a better example of four-gram similarity.")
print("Jaccard: {}   Cosine: {}".format(jaccard_distance(a,b), cosine_similarity_ngrams(a,b)))
